In [26]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# API 호출

In [151]:
# temp = pd.read_pickle('/Users/iseunghyeon/Downloads/천안예측.pickle')
# temp

# 데이터 불러오기

In [28]:
# 버스정류장 데이터 불러오기
bus = pd.read_csv('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/버스정류장/2022년_전국버스정류장 위치정보_데이터.csv')
bus.head(3)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산


### 카카오 api를 통하여, 천안시 버스정류장을 필터링하기
천안시의 버스정류장을 고려하기 위해서, 천안시의 버스정류장 데이터를 필터링한다.

제공된 버스정류장의 데이터에는 위도, 경도 좌표에 부여된 도시명과 맞지 않는 부분이 있기 때문에,

카카오 api를 활용하여, 도로명 주소를 구하고 도로명주소를 활용하여 버스 정류장이 천안시에만 있는 데이터만을 추출한다.


In [193]:
addressCopy = bus.copy()
addressCopy = addressCopy.head()
apiKey='1d49e941371e685a965fba06520cda68'
headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }

addressCopy['coord2address'] = None

cnt=0 
for idx in tqdm(range(len(addressCopy))):
  try:

    x = addressCopy.at[idx,'경도']
    y =  addressCopy.at[idx,'위도']
    searching ='x={x}&y={y}'.format(x=str(x),y=str(y))
    url='https://dapi.kakao.com/v2/local/geo/coord2address.json?{}'.format(searching)
    data = requests.post(url,headers=headers)
    result = json.loads(data.text)
    # print(result)
    documents = result['documents']
    
    addressCopy.at[idx,'coord2address'] = documents[0]
  except:
    cnt+=1
    if cnt%50==0:
      print('쿼리값을 못찾는중:',cnt) 

    pass

addressCopy

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 13.30it/s]


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
3,ASB214015459,팽성초교입구,36.952036,127.053031,20221012050123,15459.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
4,ASB214015461,대사리입구,36.944598,127.053092,20221012050123,15461.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."


In [194]:
# 제공된 버스정류장의 데이터에선 정류장번호가 ASB214010001 인 데이터의 도시명은 아산시라고 되어있지만, 카카오 api를 통해 나온 도로명 주소는 평택시로 되어있다.
# 구글 맵을 통해, 위도 경도 좌표를 넣어보면, 구글 맵또한 평택시라고 명시되어 있다.
addressCopy.loc[0,'coord2address']

{'road_address': None,
 'address': {'address_name': '경기 평택시 팽성읍 석근리 191-1',
  'region_1depth_name': '경기',
  'region_2depth_name': '평택시',
  'region_3depth_name': '팽성읍 석근리',
  'mountain_yn': 'N',
  'main_address_no': '191',
  'sub_address_no': '1',
  'zip_code': ''}}

In [29]:
# '시' 를 구분하는 region_2depth_name 데이터를 통해, 제공된 버스 정류장 데이터에 새로운 칼럼을 부여하여,
# region_2depth_name 칼럼을 통해, 천안시인 데이터만을 필터링한다.
addressCopy['region_2depth_name'] = addressCopy['coord2address'].apply(lambda elm : elm.get('address').get('region_2depth_name'))
addressCopy

NameError: name 'addressCopy' is not defined

In [ ]:
filtered_addressCopy = addressCopy[addressCopy['region_2depth_name']=='천안시']
filtered_addressCopy.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/제출용데이터dir/버스정류장_천안시.pickle')

In [30]:
bus_byApi = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/천안시 데이터 시각화 프로젝트/bus_coord2address.pickle')
bus_byApi.head(3)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9,주변시설의총합,coord2address
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 석곡...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,654,"{'road_address': None, 'address': {'address_na..."
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 석곡...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,673,"{'road_address': None, 'address': {'address_na..."
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 송화...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 남산...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 송화...,691,"{'road_address': None, 'address': {'address_na..."


In [31]:
# filtered_addressCopy = bus_byApi[['정류장번호', '정류장명', '위도', '경도', 'coord2address']]

# def get_region_2depth_name(elm):
#     if isinstance(elm, dict) and 'address' in elm:
#         address_info = elm['address']
#         return address_info.get('region_2depth_name', None)
#     return None

# filtered_addressCopy['region_2depth_name'] = filtered_addressCopy['coord2address'].apply(get_region_2depth_name)
# filtered_addressCopy = filtered_addressCopy[filtered_addressCopy['region_2depth_name'] == '천안시']
# filtered_addressCopy


,정류장번호,정류장명,위도,경도,coord2address,region_2depth_name


In [32]:
# def extract_region_name(row):
#     if row and 'address' in row and 'region_2depth_name' in row['address']:
#         return row['address']['region_2depth_name']
#     else:
#         return None  # Return a default value if keys are missing

# filtered_addressCopy['region_2depth_name'] = filtered_addressCopy['coord2address'].apply(extract_region_name)
# filtered_addressCopy = filtered_addressCopy[filtered_addressCopy['region_2depth_name'] == '천안시']
# filtered_addressCopy

,정류장번호,정류장명,위도,경도,coord2address,region_2depth_name


# P1

In [33]:
bus_station = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/버스정류장/Bus_Main.pickle')
bus_station.head(3)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,HP8,PM9,주변시설의총합,coord2address,region_2depth_name,도로명주소,시,구,읍/면/동,리
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,550,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 성환리 675-1,천안시,서북구,성환읍,성환리
1,ASB285000078,송덕,36.923783,127.118976,20221012050123,78.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,515,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 우신리 24-5,천안시,서북구,성환읍,우신리
2,ASB285000079,국립축산과학원,36.925931,127.111136,20221012050123,79.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,245,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 신방리 1-10,천안시,서북구,성환읍,신방리


### 주변시설의 총합 구하는 코드

In [38]:
CateGroupCode = ['MT1','CS2','PS3','SC4','AC5','PK6','OL7','SW8','BK9','CT1','AG2','PO3','AT4','AD5','FD6','CE7','HP8','PM9']
# CateGroupCode = ['MT1','CS2','PK6','SW8','BK9','CT1','PO3','CE7','HP8','PM9']


In [187]:
c =[elm for elm in CateGroupCode_a if elm not in CateGroupCode]
c

['PS3', 'SC4', 'AC5', 'OL7', 'AG2', 'AT4', 'AD5', 'FD6']

In [154]:

# bus_id = list(bus_station['정류장번호'].values)
# bus_station_CateGroupCode_value = pd.DataFrame({'정류장번호':bus_id})
# CateGroupCode_df = pd.DataFrame(columns=CateGroupCode)
# bus_station_CateGroupCode_value= pd.concat([bus_station_CateGroupCode_value,CateGroupCode_df],axis=1)
# # bus_station_CateGroupCode_value.head(1)

# bus_around_sum = bus_station.copy()
# bus_around_sum['filltered_총합'] = None



# for idx in tqdm(range(len(bus_around_sum))):
#     temp = 0
#     for code in CateGroupCode:
#         value = bus_around_sum.at[idx, code]
#         # value에는 딕셔너리의 값을 넣고, isinstace를 통해, value가 딕셔너리인지 확인한다. 딕셔너리가 아니라면, 딕셔너리에서 뽑을 값을 누적하지 않음.
#         if isinstance(value, dict) and 'meta' in value and 'total_count' in value['meta']:
#             # temp += value['meta']['total_count']
#              bus_station_CateGroupCode_value.at[idx,code]= value['meta']['total_count']

            
# #     bus_around_sum.at[idx, 'filltered_총합'] = temp
# # bus_around_sum[['주변시설의총합','filltered_총합']]


### 카테고리의 총합을 계산한다

In [39]:

# bus_around_sum = bus_station.copy()
bus_station['filltered_총합'] = None



for idx in tqdm(range(len(bus_station))):
    temp = 0
    for code in CateGroupCode:
        value = bus_station.at[idx, code]
        # value에는 딕셔너리의 값을 넣고, isinstace를 통해, value가 딕셔너리인지 확인한다. 딕셔너리가 아니라면, 딕셔너리에서 뽑을 값을 누적하지 않음.
        if isinstance(value, dict) and 'meta' in value and 'total_count' in value['meta']:
            temp += value['meta']['total_count']

            
    bus_station.at[idx, 'filltered_총합'] = temp
bus_station[['주변시설의총합','filltered_총합']]



100%|██████████| 2314/2314 [00:00<00:00, 2872.92it/s]


,주변시설의총합,filltered_총합
0,550,550
1,515,515
2,245,245
3,539,539
4,620,620
...,...,...
2309,620,620
2310,634,634
2311,3177,3177
2312,4943,4943


In [156]:
# bus_station_CateGroupCode_value_cp = bus_station_CateGroupCode_value.copy()
# bus_station_CateGroupCode_value_cp.fillna(bus_station_CateGroupCode_value_cp.median(),inplace=True)

# # List of columns to exclude (excluding the station number column)
# exclude_columns = ['정류장번호']

# # Convert the remaining columns to integer type
# for column in bus_station_CateGroupCode_value_cp.columns:
#     if column not in exclude_columns:
#         bus_station_CateGroupCode_value_cp[column] = bus_station_CateGroupCode_value_cp[column].astype(int)

# bus_station_CateGroupCode_value_cp.describe()

# 버스정류장과 공원사이의 거리를 구하여, 구간별로 나눈 범위중에 몇개가 있는지 구하기

In [7]:
bus_park = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시 공원정보/버스정류장과공원사이의거리.pickle')
bus_park

,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,14.919877
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.374872
2,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성소공원,"(36.81341463, 127.159059)",44131-00045,12.420054
3,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성공원(원성2),"(36.80788022, 127.1636754)",44131-00046,13.122437
4,ASB285000077,성환8리,"(36.92146807, 127.12277401)",꽃재공원(원성4),"(36.80085077, 127.1616121)",44131-00047,13.826295
...,...,...,...,...,...,...,...
765343,GGB500000184,둔포,"(36.9302833, 127.0387333)",백석공원,"(36.83303341, 127.1090661)",44133-00098,12.481557
765344,GGB500000184,둔포,"(36.9302833, 127.0387333)",두정10공원,"(36.82681344, 127.1507137)",44133-00099,15.215725
765345,GGB500000184,둔포,"(36.9302833, 127.0387333)",성성4지구 어린이공원,"(36.84053144, 127.1368524)",44133-00100,13.255707
765346,GGB500000184,둔포,"(36.9302833, 127.0387333)",문성소공원,"(36.80968397, 127.1542359)",44131-00065,16.887250


class_A : 버스정류장과 타겟과의 거리가 0이상 0.5 이하에 속하는 것

class_B : 버스정류장과 타겟과의 거리가 0.5초과 1 이하에 속하는 것

(거리 단위는 km)

In [9]:
# 버스 정류장과 공원 사이의 거리가 0<=distance<=0.5, 0.5<distance<=1 의 갯수를 각각 구한다.
bus_park_class_A = bus_park[(0<=bus_park['distance'])&(bus_park['distance']<=0.5)]
bus_park_class_B = bus_park[(0.5<bus_park['distance'])&(bus_park['distance']<=1)]
bus_park_class_A.head(2)
bus_park_class_B.head(2)


,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
1493,ASB285000099,삼풍아파트,"(36.9125166, 127.1309215)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.317816
2933,ASB285000165,쌍용동일아파트,"(36.7848, 127.11328)",쌍용10공원,"(36.78655184, 127.1168556)",44133-00077,0.373709


,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
785,ASB285000095,성환터미널,"(36.91722136, 127.13273235)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.568513
962,ASB285000096,성신초등학교,"(36.9176694, 127.1277947)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.842896


In [10]:
# 공원거리구분_A : 0.5<distance<=1 
class_A_df= pd.DataFrame(bus_park_class_A['정류장번호'].value_counts()).reset_index()
class_A_df.columns =['정류장번호','공원거리구분_A']
class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
class_A_df.reset_index(drop=True,inplace=True)
class_A_df

# 공원거리구분_B : 0.5<distance<=1 

# class_B_busId = list(bus_park_class_B.groupby('정류장번호').count().reset_index()['정류장번호'].values)
# class_B_count = list(bus_park_class_B.groupby('정류장번호').count().reset_index()['distance'].values)
# class_B_df= pd.DataFrame({'정류장번호':class_B_busId,'공원거리구분_B':class_B_count})
# class_B_df

class_B_df= pd.DataFrame(bus_park_class_B['정류장번호'].value_counts()).reset_index()
class_B_df.columns =['정류장번호','공원거리구분_B']
class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
class_B_df.reset_index(drop=True,inplace=True)
class_B_df

,정류장번호,공원거리구분_A
0,ASB285000099,1
1,ASB285000165,1
2,ASB285000287,1
3,ASB285000649,2
4,ASB285000650,2
...,...,...
781,GGB277102211,1
782,GGB285000034,1
783,GGB285500010,4
784,GGB285500020,6


,정류장번호,공원거리구분_B
0,ASB285000095,1
1,ASB285000096,1
2,ASB285000097,1
3,ASB285000098,1
4,ASB285000100,1
...,...,...
1008,GGB285500010,4
1009,GGB285500020,8
1010,GGB285500030,8
1011,GGB288500130,8


In [11]:
bus_station_merge_class_A = pd.merge(bus_station,class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print(len(bus_station)==len(bus_station_merge_class_A))

bus_station_merge_class_AB = pd.merge(bus_station_merge_class_A,class_B_df, on='정류장번호',how='left')


print(len(bus_station)==len(bus_station_merge_class_AB))

True
True


In [12]:
bus_station_merge_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_class_AB['공원거리구분_A'] = bus_station_merge_class_AB['공원거리구분_A'].fillna(0)
bus_station_merge_class_AB['공원거리구분_B'] = bus_station_merge_class_AB['공원거리구분_B'].fillna(0)

bus_station_merge_class_AB['공원거리구분_A'] = bus_station_merge_class_AB['공원거리구분_A'].astype('int')
bus_station_merge_class_AB['공원거리구분_B'] = bus_station_merge_class_AB['공원거리구분_B'].astype('int')

bus_station_merge_class_AB.head(1)


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,주변시설의총합,coord2address,region_2depth_name,도로명주소,시,구,읍/면/동,리,공원거리구분_A,공원거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,550,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 성환리 675-1,천안시,서북구,성환읍,성환리,NaN,NaN


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,주변시설의총합,coord2address,region_2depth_name,도로명주소,시,구,읍/면/동,리,공원거리구분_A,공원거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,550,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 성환리 675-1,천안시,서북구,성환읍,성환리,0,0


# 버스정류장과 거주지,회사 의 갯수

위의 작업과 마찬가지로, 버스정류장과 거주지,회사간의 거리를 구하여 그 거리를 구간별로 나눠 갯수를 구하는 작업을 진행한다.

In [13]:
bus_CH = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/거리계산/bus_Around_company_house.pickle')
bus_CH.head(2)

,정류장번호,정류장명,coord2address,주변시설의총합,location,이름,byAPI,house_loc,종류,distance
0,ASB285000077,성환8리,"{'road_address': None, 'address': {'address_na...",550,"(36.92146807, 127.12277401)",(주)동아수지산업,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8656506084092, 127.103428417573)",제조업체,6.429916
1,ASB285000077,성환8리,"{'road_address': None, 'address': {'address_na...",550,"(36.92146807, 127.12277401)",KY컴퍼니,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8662931269303, 127.104524303521)",제조업체,6.335470


## A,B 로 구간 나누기

In [14]:
bus_CH_A = bus_CH[(0<=bus_CH['distance'])&(bus_CH['distance']<=0.5)]
bus_CH_B = bus_CH[(0.5<bus_CH['distance'])&(bus_CH['distance']<=1)]



In [15]:
# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
# bus_CH_class_A_busId = list(bus_CH_A.groupby('정류장번호').count().reset_index()['정류장번호'].values)
# bus_CH_class_A_count = list(bus_CH_A.groupby('정류장번호').count().reset_index()['distance'].values)
# bus_CH_class_A_df= pd.DataFrame({'정류장번호':bus_CH_class_A_busId,'CH거리구분_A':bus_CH_class_A_count})
# bus_CH_class_A_df.reset_index(drop=True,inplace=True)
# bus_CH_class_A_df.head(2)



# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5< distance<=1
# bus_CH_class_B_busId = list(bus_CH_B.groupby('정류장번호').count().reset_index()['정류장번호'].values)
# bus_CH_class_B_count = list(bus_CH_B.groupby('정류장번호').count().reset_index()['distance'].values)
# bus_CH_class_B_df= pd.DataFrame({'정류장번호':bus_CH_class_B_busId,'CH거리구분_B':bus_CH_class_B_count})
# bus_CH_class_B_df.head(2)



# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
bus_CH_class_A_df= pd.DataFrame(bus_CH_A['정류장번호'].value_counts()).reset_index()
bus_CH_class_A_df.columns =['정류장번호','CH거리구분_A']
bus_CH_class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_CH_class_A_df.reset_index(drop=True,inplace=True)
bus_CH_class_A_df.head(2)

# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5< distance<=1
bus_CH_class_B_df= pd.DataFrame(bus_CH_B['정류장번호'].value_counts()).reset_index()
bus_CH_class_B_df.columns =['정류장번호','CH거리구분_B']
bus_CH_class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_CH_class_B_df.reset_index(drop=True,inplace=True)
bus_CH_class_B_df.head(2)

,정류장번호,CH거리구분_A
0,ASB285000077,5
1,ASB285000078,11


,정류장번호,CH거리구분_B
0,ASB285000077,25
1,ASB285000078,15


In [16]:
# 테이블을 병합하기위해, copy본과 결합
bus_station_merge_class_AB_cp = bus_station_merge_class_AB.copy()

bus_station_merge_CH_class_A = pd.merge(bus_station_merge_class_AB_cp,bus_CH_class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_CH_class_A))

bus_station_merge_CH_class_AB = pd.merge(bus_station_merge_CH_class_A,bus_CH_class_B_df, on='정류장번호',how='left')

print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_CH_class_AB))

bus_station_merge_CH_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_CH_class_AB['CH거리구분_A'] = bus_station_merge_CH_class_AB['CH거리구분_A'].fillna(0)
bus_station_merge_CH_class_AB['CH거리구분_B'] = bus_station_merge_CH_class_AB['CH거리구분_B'].fillna(0)

bus_station_merge_CH_class_AB['CH거리구분_A'] = bus_station_merge_CH_class_AB['CH거리구분_A'].astype('int')
bus_station_merge_CH_class_AB['CH거리구분_B'] = bus_station_merge_CH_class_AB['CH거리구분_B'].astype('int')

# 병합된 데이터 셋
bus_station_merge_CH_class_AB.head(1)


테이블간 결합됨 True
테이블간 결합됨 True


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,region_2depth_name,도로명주소,시,구,읍/면/동,리,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,천안시 서북구,충남 천안시 서북구 성환읍 성환리 675-1,천안시,서북구,성환읍,성환리,0,0,5.0,25.0


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,region_2depth_name,도로명주소,시,구,읍/면/동,리,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,천안시 서북구,충남 천안시 서북구 성환읍 성환리 675-1,천안시,서북구,성환읍,성환리,0,0,5,25


# 버스정류장과 하천과의 거리 갯수

위의 작업들과 동일하며, 버스정류장과 하천과의 거리를 구하여 구간별로 나눠 갯수를 센다

In [17]:
bus_stream = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/하천정보/버스정류장과하천과의거리.pickle')
bus_stream.head(2)

,정류장번호,정류장명,location,A1,하천좌표,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001030,"(35.8484069199, 127.9088593132)",138.392944
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001031,"(35.841851007, 127.9084686265)",139.003014


In [18]:
bus_stream_A  = bus_stream[(0<=bus_stream['distance'])&(bus_stream['distance']<=0.5)]
bus_stream_B  = bus_stream[(0.5<bus_stream['distance'])&(bus_stream['distance']<=1)]
bus_stream_A.head(2)
bus_stream_B.head(2)


,정류장번호,정류장명,location,A1,하천좌표,distance
7736,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.398270
37124,ASB285000094,성환8리,"(36.9214794, 127.1225049)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.421355


,정류장번호,정류장명,location,A1,하천좌표,distance
17532,ASB285000078,송덕,"(36.92378302, 127.11897568)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.813301
46920,ASB285000095,성환터미널,"(36.91722136, 127.13273235)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.613733


In [19]:
# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
# bus_stream_class_A_busId = list(bus_stream_A.groupby('정류장번호').count().reset_index()['정류장번호'].values)
# bus_stream_class_A_count = list(bus_stream_A.groupby('정류장번호').count().reset_index()['distance'].values)
# bus_stream_class_A_df= pd.DataFrame({'정류장번호':bus_stream_class_A_busId,'하천거리구분_A':bus_stream_class_A_count})
# bus_stream_class_A_df.head(2)

# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5<distance<=1
# bus_stream_class_B_busId = list(bus_stream_B.groupby('정류장번호').count().reset_index()['정류장번호'].values)
# bus_stream_class_B_count = list(bus_stream_B.groupby('정류장번호').count().reset_index()['distance'].values)
# bus_stream_class_B_df= pd.DataFrame({'정류장번호':bus_stream_class_B_busId,'하천거리구분_B':bus_stream_class_B_count})
# bus_stream_class_B_df.head(2)


# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
bus_stream_class_A_df= pd.DataFrame(bus_stream_A['정류장번호'].value_counts()).reset_index()
bus_stream_class_A_df.columns =['정류장번호','하천거리구분_A']
bus_stream_class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_stream_class_A_df.reset_index(drop=True,inplace=True)
bus_stream_class_A_df.head(2)

# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5<distance<=1
bus_stream_class_B_df= pd.DataFrame(bus_stream_B['정류장번호'].value_counts()).reset_index()
bus_stream_class_B_df.columns =['정류장번호','하천거리구분_B']
bus_stream_class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_stream_class_B_df.reset_index(drop=True,inplace=True)
bus_stream_class_B_df.head(2)

,정류장번호,하천거리구분_A
0,ASB285000077,1
1,ASB285000094,1


,정류장번호,하천거리구분_B
0,ASB285000078,1
1,ASB285000095,1


In [20]:
# 테이블을 병합하기위해, copy본과 결합
bus_station_merge_class_AB_cp = bus_station_merge_CH_class_AB.copy()

bus_station_merge_stream_class_A = pd.merge(bus_station_merge_class_AB_cp,bus_stream_class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_stream_class_A))

bus_station_merge_stream_class_AB = pd.merge(bus_station_merge_stream_class_A,bus_stream_class_B_df, on='정류장번호',how='left')

print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_stream_class_AB))

bus_station_merge_stream_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_stream_class_AB['하천거리구분_A'] = bus_station_merge_stream_class_AB['하천거리구분_A'].fillna(0)
bus_station_merge_stream_class_AB['하천거리구분_B'] = bus_station_merge_stream_class_AB['하천거리구분_B'].fillna(0)

bus_station_merge_stream_class_AB['하천거리구분_A'] = bus_station_merge_stream_class_AB['하천거리구분_A'].astype('int')
bus_station_merge_stream_class_AB['하천거리구분_B'] = bus_station_merge_stream_class_AB['하천거리구분_B'].astype('int')

bus_station_merge_stream_class_AB.head(1)


테이블간 결합됨 True
테이블간 결합됨 True


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,시,구,읍/면/동,리,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,천안시,서북구,성환읍,성환리,0,0,5,25,1.0,NaN


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,시,구,읍/면/동,리,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,천안시,서북구,성환읍,성환리,0,0,5,25,1,0


In [21]:
bus_station_merge_stream_class_AB.head()

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,시,구,읍/면/동,리,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,천안시,서북구,성환읍,성환리,0,0,5,25,1,0
1,ASB285000078,송덕,36.923783,127.118976,20221012050123,78.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,천안시,서북구,성환읍,우신리,0,0,11,15,0,1
2,ASB285000079,국립축산과학원,36.925931,127.111136,20221012050123,79.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,천안시,서북구,성환읍,신방리,0,0,12,22,0,0
3,ASB285000094,성환8리,36.921479,127.122505,20221012050123,94.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,천안시,서북구,성환읍,성환리,0,0,5,24,1,0
4,ASB285000095,성환터미널,36.917221,127.132732,20221012050123,95.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,천안시,서북구,성환읍,성환리,0,1,11,16,0,1


In [24]:
bus_station_merge_stream_class_AB.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/버스정류장/Bus_Main0823.pickle')

In [67]:
bus_station_merge_stream_class_AB.loc[0,'FD6']

{'documents': [{'address_name': '충남 천안시 서북구 성환읍 송덕리 54-3',
   'category_group_code': 'FD6',
   'category_group_name': '음식점',
   'category_name': '음식점 > 한식 > 육류,고기 > 닭요리',
   'distance': '425',
   'id': '11032502',
   'phone': '041-581-2037',
   'place_name': '공주집',
   'place_url': 'http://place.map.kakao.com/11032502',
   'road_address_name': '충남 천안시 서북구 성환읍 송덕2길 113',
   'x': '127.118845426498',
   'y': '36.9236494148593'},
  {'address_name': '충남 천안시 서북구 성환읍 성환리 449-19',
   'category_group_code': 'FD6',
   'category_group_name': '음식점',
   'category_name': '음식점 > 중식 > 중국요리',
   'distance': '799',
   'id': '9780807',
   'phone': '041-581-2070',
   'place_name': '동순원 성환본점',
   'place_url': 'http://place.map.kakao.com/9780807',
   'road_address_name': '충남 천안시 서북구 성환읍 성환중앙로 33',
   'x': '127.12980936605447',
   'y': '36.91698966919916'},
  {'address_name': '충남 천안시 서북구 성환읍 성환리 451-5',
   'category_group_code': 'FD6',
   'category_group_name': '음식점',
   'category_name': '음식점 > 한식 > 순대',
   '

# Score 계산하기

버스정류장의 반경 2km 이내에 있는 각 카테고리(마트,편의점,병원,,)의 수 를 집계하여

구분 A의 값은 2배의 값을 곱하여 더하고,

구분 B의 값은 자기 자신의 값을 더하여 score 칼럼을 구한다.

(score 칼럼 : 버스정류장의 중요도를 나타내는 정도)

In [68]:
score_df = bus_station_merge_stream_class_AB.copy()

In [69]:
score_df.columns

Index(['정류장번호', '정류장명', '위도', '경도', '정보수집일시', '모바일단축번호', '도시코드', '도시명',
       '관리도시명', 'MT1', 'CS2', 'PS3', 'SC4', 'AC5', 'PK6', 'OL7', 'SW8', 'BK9',
       'CT1', 'AG2', 'PO3', 'AT4', 'AD5', 'FD6', 'CE7', 'HP8', 'PM9',
       '주변시설의총합', 'coord2address', 'region_2depth_name', '도로명주소', '시', '구',
       '읍/면/동', '리', 'filltered_총합', '공원거리구분_A', '공원거리구분_B', 'CH거리구분_A',
       'CH거리구분_B', '하천거리구분_A', '하천거리구분_B'],
      dtype='object')

In [84]:
score_df['CH거리구분_A'].describe()


count    2314.000000
mean       14.159896
std        37.193543
min         0.000000
25%         1.000000
50%         5.000000
75%        11.750000
max       459.000000
Name: CH거리구분_A, dtype: float64

In [87]:

# 카테고리의 갯수를 구하는 코드
from tqdm import tqdm
CateGroupCode = ['MT1','CS2','PS3','SC4','AC5','PK6','OL7','SW8','BK9','CT1','AG2','PO3','AT4','AD5','FD6','CE7','HP8','PM9']

temp_list = []
bus_count_category = score_df.copy()
bus_count_category.reset_index(drop=True,inplace=True)
for idx in tqdm(range(len(bus_count_category))):
    data = {
        '정류장번호': bus_count_category.at[idx, '정류장번호'],
        '정류장명': bus_count_category.at[idx, '정류장명']
    }

    for category in CateGroupCode:
        category_data = bus_count_category.at[idx, category]

        if isinstance(category_data, dict):
            total_count = category_data.get('meta', {}).get('total_count', 0)
            data[category] = total_count
        else:
            data[category] = 0  # Set to 0 if the value is not a dictionary

    temp_list.append(data)

bus_count_category_df = pd.DataFrame(temp_list)
bus_count_category_df


100%|██████████| 2314/2314 [00:00<00:00, 13051.10it/s]


,정류장번호,정류장명,MT1,CS2,PS3,SC4,AC5,PK6,OL7,SW8,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,ASB285000077,성환8리,4,23,8,5,33,8,9,1,19,3,54,4,6,6,291,44,21,11
1,ASB285000078,송덕,4,20,7,4,31,8,8,1,16,3,53,4,4,6,272,42,21,11
2,ASB285000079,국립축산과학원,1,10,3,2,8,4,3,1,12,2,29,3,5,5,116,24,11,6
3,ASB285000094,성환8리,4,22,8,0,33,8,9,1,18,3,54,4,6,6,288,43,21,11
4,ASB285000095,성환터미널,4,28,12,6,36,16,11,1,21,4,54,4,4,6,326,54,21,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,GGB285000034,성환터미널,4,28,12,6,36,16,11,1,21,4,54,4,4,6,326,54,21,12
2310,GGB285000035,남서울대학,4,28,12,6,37,15,10,1,22,4,52,2,5,6,339,56,21,14
2311,GGB285500010,천안TG.단대.천안대,7,154,83,20,213,99,21,2,110,21,303,8,7,134,1449,384,122,40
2312,GGB285500020,천안종합터미널,9,222,109,23,310,127,22,2,163,24,507,15,10,211,2433,508,181,67


#######################################################################################################################################

In [70]:
# score 칼럼 구하기
score_df['score'] = score_df['filltered_총합'].astype('int') + (score_df['공원거리구분_A']*2 + score_df['공원거리구분_B']) + (score_df['CH거리구분_A'] *2 + score_df['CH거리구분_B']) + (score_df['하천거리구분_A'] *2 + score_df['하천거리구분_B'])
# score_df[['주변시설의총합','score']]
sorted_score_df =score_df[['정류장번호','정류장명','위도','경도','도로명주소','시','구','읍/면/동','리','score']].sort_values(by='score',ascending=False)
sorted_score_df


,정류장번호,정류장명,위도,경도,도로명주소,시,구,읍/면/동,리,score
844,CAB285000924,"성정2동행정복지센터,건강생활지원센터",36.819458,127.139441,충남 천안시 서북구 성정동 762,천안시,서북구,성정동,None,2053
52,ASB285000880,가구웨딩특화거리,36.818650,127.139030,충남 천안시 서북구 성정동 682-14,천안시,서북구,성정동,None,2049
804,CAB285000880,가구웨딩특화거리,36.818650,127.139033,충남 천안시 서북구 성정동 682-14,천안시,서북구,성정동,None,2049
843,CAB285000923,"성정2동행정복지센터,건강생활지원센터",36.819683,127.139405,충남 천안시 서북구 성정동 680-4,천안시,서북구,성정동,None,2047
51,ASB285000879,가구웨딩특화거리,36.818490,127.139680,충남 천안시 서북구 성정동 750,천안시,서북구,성정동,None,2045
...,...,...,...,...,...,...,...,...,...,...
1450,CAB285001737,관성리(종점),36.817087,127.340944,충남 천안시 동남구 병천면 관성리 207-1,천안시,동남구,병천면,관성리,0
1589,CAB285001964,성결원입구,36.799323,127.352820,충남 천안시 동남구 동면 행암리 273-1,천안시,동남구,동면,행암리,0
1240,CAB285001450,관성3리회관,36.810731,127.332556,충남 천안시 동남구 병천면 관성리 412-1,천안시,동남구,병천면,관성리,0
1241,CAB285001451,관성리(종점),36.817172,127.340865,충남 천안시 동남구 병천면 관성리 207-5,천안시,동남구,병천면,관성리,0


In [74]:
sorted_score_df.to_csv('/Users/iseunghyeon/Desktop/프로젝트/data/최종데이터/scoring_result_0823_1341.csv',index=False)

In [75]:
t = pd.read_csv('/Users/iseunghyeon/Desktop/프로젝트/data/최종데이터/scoring_result_0823_1341.csv')
t

,정류장번호,정류장명,위도,경도,도로명주소,시,구,읍/면/동,리,score
0,CAB285000924,"성정2동행정복지센터,건강생활지원센터",36.819458,127.139441,충남 천안시 서북구 성정동 762,천안시,서북구,성정동,NaN,2053
1,ASB285000880,가구웨딩특화거리,36.818650,127.139030,충남 천안시 서북구 성정동 682-14,천안시,서북구,성정동,NaN,2049
2,CAB285000880,가구웨딩특화거리,36.818650,127.139033,충남 천안시 서북구 성정동 682-14,천안시,서북구,성정동,NaN,2049
3,CAB285000923,"성정2동행정복지센터,건강생활지원센터",36.819683,127.139405,충남 천안시 서북구 성정동 680-4,천안시,서북구,성정동,NaN,2047
4,ASB285000879,가구웨딩특화거리,36.818490,127.139680,충남 천안시 서북구 성정동 750,천안시,서북구,성정동,NaN,2045
...,...,...,...,...,...,...,...,...,...,...
2309,CAB285001737,관성리(종점),36.817087,127.340944,충남 천안시 동남구 병천면 관성리 207-1,천안시,동남구,병천면,관성리,0
2310,CAB285001964,성결원입구,36.799323,127.352820,충남 천안시 동남구 동면 행암리 273-1,천안시,동남구,동면,행암리,0
2311,CAB285001450,관성3리회관,36.810731,127.332556,충남 천안시 동남구 병천면 관성리 412-1,천안시,동남구,병천면,관성리,0
2312,CAB285001451,관성리(종점),36.817172,127.340865,충남 천안시 동남구 병천면 관성리 207-5,천안시,동남구,병천면,관성리,0


In [71]:
sorted_score_df['도로명주소'].value_counts()

충남 천안시 서북구 성정동 749           18
충남 천안시 서북구 쌍용동 1899          10
충남 천안시 서북구 불당동 1446           8
충남 천안시 서북구 성정동 748            8
충남 천안시 서북구 불당동 1442           7
                             ..
충남 천안시 서북구 직산읍 자은가리 53-12     1
충남 천안시 동남구 수신면 장산리 852-8      1
충남 천안시 동남구 수신면 장산리 706-1      1
충남 천안시 동남구 병천면 가전리 379-1      1
충남 천안시 동남구 동면 행암리 272-3       1
Name: 도로명주소, Length: 1895, dtype: int64

In [180]:
sorted_score_df.describe()

,위도,경도,score
count,2314.000000,2314.000000,2314.000000
mean,36.816894,127.174281,442.775281
std,0.069034,0.070452,573.967158
min,36.650342,127.025892,0.000000
25%,36.770936,127.124439,33.000000
50%,36.811284,127.151465,110.500000
75%,36.867183,127.217029,895.000000
max,36.966230,127.418926,2053.000000


In [45]:
# # 정류장명이 중복인게 있다.
# sorted_score_df['정류장명'].value_counts()
# sorted_score_df[sorted_score_df['정류장명']=='석곡리']

# 클러스터링 진행

## 클러스터 갯수 구하기

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

data = cutted_sorted_score_df.copy()
coordinates = data[['위도', '경도']]

# Determine a range of cluster numbers to try
num_clusters_range = range(1, 31)  # You can adjust the range as needed

inertias = []  # To store the inertia values

# Iterate through different cluster numbers and calculate inertia
for num_clusters in num_clusters_range:
    kmeans = KMeans(n_clusters=num_clusters, random_state=98)
    kmeans.fit(coordinates)
    inertias.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(num_clusters_range, inertias, marker='o')
plt.title('Elbow Curve')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()


In [ ]:
from matplotlib import pyplot as plt    
plt.figure(figsize=(10, 4))

plt.plot(kn, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')

plt.xticks([0,20,40,60,80,100,400])
plt.show()

#최적 클러스터 수 확인 -> 10개
from kneed import KneeLocator

kl = KneeLocator(range(1,len(inertias)+1), inertias, curve='convex', direction='decreasing')
optimal_k = kl.elbow
print(optimal_k)

엘보우 포인트와 실루엣 계수에서 나온 최적의 클러스터 값으로 

클러스터 갯수 설정하기

In [ ]:
from sklearn.cluster import KMeans
coordinates = sorted_score_df[['위도', '경도']]
num_clusters = 21 
kmeans = KMeans(n_clusters=num_clusters,random_state=98)
kmeans.fit(coordinates)

sorted_score_df['cluster'] = kmeans.labels_
print("Cluster Centers:")
print(kmeans.cluster_centers_)
print(data['cluster'].value_counts())

# 중요한 버스 정류장 선별하기

회귀분석 모델에서 나온 결과값 : 188을 이용하여,

위에서 집계산 score칼럼을 내림차순을로 정렬하고 회귀분석 결과값을 기준 잘라서 

천안시에 버스정류장 주변에 따릉이를 설치할 버스정류장을 선별한다.

In [76]:
cutted_df = sorted_score_df
cutted_df

,정류장번호,정류장명,위도,경도,도로명주소,시,구,읍/면/동,리,score
844,CAB285000924,"성정2동행정복지센터,건강생활지원센터",36.819458,127.139441,충남 천안시 서북구 성정동 762,천안시,서북구,성정동,None,2053
52,ASB285000880,가구웨딩특화거리,36.818650,127.139030,충남 천안시 서북구 성정동 682-14,천안시,서북구,성정동,None,2049
804,CAB285000880,가구웨딩특화거리,36.818650,127.139033,충남 천안시 서북구 성정동 682-14,천안시,서북구,성정동,None,2049
843,CAB285000923,"성정2동행정복지센터,건강생활지원센터",36.819683,127.139405,충남 천안시 서북구 성정동 680-4,천안시,서북구,성정동,None,2047
51,ASB285000879,가구웨딩특화거리,36.818490,127.139680,충남 천안시 서북구 성정동 750,천안시,서북구,성정동,None,2045
...,...,...,...,...,...,...,...,...,...,...
1450,CAB285001737,관성리(종점),36.817087,127.340944,충남 천안시 동남구 병천면 관성리 207-1,천안시,동남구,병천면,관성리,0
1589,CAB285001964,성결원입구,36.799323,127.352820,충남 천안시 동남구 동면 행암리 273-1,천안시,동남구,동면,행암리,0
1240,CAB285001450,관성3리회관,36.810731,127.332556,충남 천안시 동남구 병천면 관성리 412-1,천안시,동남구,병천면,관성리,0
1241,CAB285001451,관성리(종점),36.817172,127.340865,충남 천안시 동남구 병천면 관성리 207-5,천안시,동남구,병천면,관성리,0


In [78]:
ctc =['도로명주소']
ctc_df = sorted_score_df.drop_duplicates(subset = ctc, keep='first')
# ctc_df = ctc_df.head(188)
ctc_df

,정류장번호,정류장명,위도,경도,도로명주소,시,구,읍/면/동,리,score
844,CAB285000924,"성정2동행정복지센터,건강생활지원센터",36.819458,127.139441,충남 천안시 서북구 성정동 762,천안시,서북구,성정동,None,2053
52,ASB285000880,가구웨딩특화거리,36.818650,127.139030,충남 천안시 서북구 성정동 682-14,천안시,서북구,성정동,None,2049
843,CAB285000923,"성정2동행정복지센터,건강생활지원센터",36.819683,127.139405,충남 천안시 서북구 성정동 680-4,천안시,서북구,성정동,None,2047
51,ASB285000879,가구웨딩특화거리,36.818490,127.139680,충남 천안시 서북구 성정동 750,천안시,서북구,성정동,None,2045
68,ASB285000907,성정1동주민센터,36.817311,127.142562,충남 천안시 서북구 성정동 314-2,천안시,서북구,성정동,None,1989
...,...,...,...,...,...,...,...,...,...,...
1450,CAB285001737,관성리(종점),36.817087,127.340944,충남 천안시 동남구 병천면 관성리 207-1,천안시,동남구,병천면,관성리,0
1589,CAB285001964,성결원입구,36.799323,127.352820,충남 천안시 동남구 동면 행암리 273-1,천안시,동남구,동면,행암리,0
1240,CAB285001450,관성3리회관,36.810731,127.332556,충남 천안시 동남구 병천면 관성리 412-1,천안시,동남구,병천면,관성리,0
1241,CAB285001451,관성리(종점),36.817172,127.340865,충남 천안시 동남구 병천면 관성리 207-5,천안시,동남구,병천면,관성리,0


In [79]:
ctc_df.reset_index(drop=True,inplace=True)
ctc_df

,정류장번호,정류장명,위도,경도,도로명주소,시,구,읍/면/동,리,score
0,CAB285000924,"성정2동행정복지센터,건강생활지원센터",36.819458,127.139441,충남 천안시 서북구 성정동 762,천안시,서북구,성정동,None,2053
1,ASB285000880,가구웨딩특화거리,36.818650,127.139030,충남 천안시 서북구 성정동 682-14,천안시,서북구,성정동,None,2049
2,CAB285000923,"성정2동행정복지센터,건강생활지원센터",36.819683,127.139405,충남 천안시 서북구 성정동 680-4,천안시,서북구,성정동,None,2047
3,ASB285000879,가구웨딩특화거리,36.818490,127.139680,충남 천안시 서북구 성정동 750,천안시,서북구,성정동,None,2045
4,ASB285000907,성정1동주민센터,36.817311,127.142562,충남 천안시 서북구 성정동 314-2,천안시,서북구,성정동,None,1989
...,...,...,...,...,...,...,...,...,...,...
1890,CAB285001737,관성리(종점),36.817087,127.340944,충남 천안시 동남구 병천면 관성리 207-1,천안시,동남구,병천면,관성리,0
1891,CAB285001964,성결원입구,36.799323,127.352820,충남 천안시 동남구 동면 행암리 273-1,천안시,동남구,동면,행암리,0
1892,CAB285001450,관성3리회관,36.810731,127.332556,충남 천안시 동남구 병천면 관성리 412-1,천안시,동남구,병천면,관성리,0
1893,CAB285001451,관성리(종점),36.817172,127.340865,충남 천안시 동남구 병천면 관성리 207-5,천안시,동남구,병천면,관성리,0


In [236]:
# pip install folium

     |████████████████████████████████| 102 kB 5.9 MB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [237]:
import folium

In [ ]:
import pandas as pd
import folium

# Load your DataFrame (ctc_df)
# ctc_df = pd.read_csv('your_data.csv')

# Convert latitude and longitude columns to float if needed
ctc_df['위도'] = ctc_df['위도'].astype(float)
ctc_df['경도'] = ctc_df['경도'].astype(float)

# Calculate the mean latitude and longitude for centering the map
center_lat = ctc_df['위도'].mean()
center_lon = ctc_df['경도'].mean()

# Create a Folium map centered at the mean location
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers for each latitude and longitude pair in your DataFrame
for index, row in ctc_df.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['정류장명'],  # Optional popup information
        icon=folium.Icon(color='blue')  # Optional icon customization
    ).add_to(m)

# Save the map to an HTML file
m.save('./map2.html')


In [ ]:
merge1 = pd.merge(bus_count_category_df,score_df.loc[:,['정류장번호','공원거리구분_A','공원거리구분_B','CH거리구분_A','CH거리구분_B','하천거리구분_A','하천거리구분_B']],on='정류장번호',how='left')
merge1
scalling_cols= list(merge1.columns[2:])
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(merge1[scalling_cols])
scaled_df = pd.DataFrame(scaled_data,columns=scalling_cols)
merge1[scalling_cols] = scaled_df
# scaled_df
merge1.head(2)
merge1['filltered_총합'] = merge1.loc[:,'MT1':'PM9'].sum(axis=1)
score_df = merge1.copy()
# score 칼럼 구하기
score_df['score'] = score_df['filltered_총합'] + (score_df['공원거리구분_A']*2 + score_df['공원거리구분_B']) + (score_df['CH거리구분_A'] *2 + score_df['CH거리구분_B']) + (score_df['하천거리구분_A'] *2 + score_df['하천거리구분_B'])
# score_df[['주변시설의총합','score']]
sorted_score_df = score_df.sort_values(by='score',ascending=False)
score_df.head(2)

In [138]:
score_df.head(1)

,정류장번호,정류장명,MT1,CS2,PS3,SC4,AC5,PK6,OL7,SW8,...,HP8,PM9,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,filltered_총합,score
0,ASB285000077,성환8리,0.266667,0.0721,0.037209,0.138889,0.036105,0.04908,0.3,0.333333,...,0.073944,0.104762,0.0,0.0,0.010893,0.0499,0.333333,0.0,2.083235,2.821588


In [137]:
bus_loc = bus_station[['정류장번호','위도','경도']]

In [139]:
score_df_1 = pd.merge(score_df,bus_loc,on='정류장번호',how='left')
score_df_1.head(1)

,정류장번호,정류장명,MT1,CS2,PS3,SC4,AC5,PK6,OL7,SW8,...,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,filltered_총합,score,위도,경도
0,ASB285000077,성환8리,0.266667,0.0721,0.037209,0.138889,0.036105,0.04908,0.3,0.333333,...,0.0,0.0,0.010893,0.0499,0.333333,0.0,2.083235,2.821588,36.921468,127.122774


In [ ]:
ctc_df = pd.read_csv('/Users/iseunghyeon/Downloads/scoring_result_0823_1341_top25p_cluster188.csv')
# ctc_df = score_df_1.copy()
# ctc_df = ctc_df.head(188)
# Convert latitude and longitude columns to float if needed

ctc_df.rename(columns={'0':'위도','1':'경도'},inplace=True)

ctc_df['위도'] = ctc_df['위도'].astype(float)
ctc_df['경도'] = ctc_df['경도'].astype(float)

# Calculate the mean latitude and longitude for centering the map
center_lat = ctc_df['위도'].mean()
center_lon = ctc_df['경도'].mean()

# Create a Folium map centered at the mean location
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers for each latitude and longitude pair in your DataFrame
for index, row in ctc_df.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        # popup=row['정류장명'],  # Optional popup information
        icon=folium.Icon(color='blue')  # Optional icon customization
    ).add_to(m)

# Save the map to an HTML file
m.save('./map4.html')

In [145]:
temp

,Unnamed: 0,0,1,도시명
0,0,36.815998,127.112622,천안시
1,1,36.806265,127.155321,천안시
2,2,36.817016,127.135254,천안시
3,3,36.793285,127.122715,천안시
4,4,36.837595,127.138252,천안시
...,...,...,...,...
183,183,36.834522,127.138197,천안시
184,184,36.806646,127.111769,천안시
185,185,36.821967,127.106829,천안시
186,186,36.805142,127.116411,천안시
